In [1]:
import os
import logging
import json
import pickle
from tqdm import tqdm
import glob
import re
from itertools import chain
import numpy as np
import pandas as pd
import collections
import seaborn as sns
import matplotlib.pyplot as plt

# Document Retrieval

In [2]:
try:
    # load dir_dict from json file in home directory
    home_dir = os.path.expanduser("~")
    with open(f"{home_dir}/.biomedqa_dir.json", encoding="utf-8") as fp:
        dir_dict = json.load(fp)
except Exception as exc:
    print("Error: unable to load directory dictionary. Please run setup.py")
    raise exc

# set directories
BASE_DIR = dir_dict["base_dir"]
DATA_DIR = dir_dict["data_dir"]
MODEL_DIR = dir_dict["model_dir"]
LOG_DIR = dir_dict["log_dir"]
RESULTS_DIR = dir_dict["results_dir"]


# set parameters
challenge_no = 10
DATASET = "bioasq"
YEAR = "2022"
DATASET_NAME = "Task10BGoldenEnriched"
RESULT_FILE_NAME = "lda_Task10BGoldenEnriched_on_Task10BGoldenEnriched_CmaEsSampler_v1.csv"
RESULTS_DIR = f"{RESULTS_DIR}/biomed_qa/document_retrieval/topic_model/{DATASET}/{YEAR}/{DATASET_NAME}"

# Load data from results_file into a DataFrame
results_df = pd.read_csv(f"{RESULTS_DIR}/{RESULT_FILE_NAME}")
display(results_df.head(10))
# set challenge & metrics
challenge = f"BioASQ Task {challenge_no}b - Phase A"
metrics = results_df.columns[1:-1]
display(metrics)

,System,Mean precision,Recall,F-Measure,Batch
0,Proposed,0.305556,0.540372,0.312666,1
1,bio-answerfinder,0.390800,0.417000,0.355300,1
2,RYGH-1,0.288900,0.612200,0.299900,1
3,RYGH-4,0.277400,0.617700,0.294300,1
4,RYGH-3,0.276500,0.616200,0.293700,1
5,RYGH,0.273000,0.598000,0.291100,1
6,The basic end-to-end,0.249600,0.513500,0.278700,1
7,Basic e2e mid speed,0.239600,0.496000,0.266800,1
8,gsl_zs_rrf1,0.231100,0.565800,0.258400,1
9,bio-answerfinder-2,0.261300,0.471500,0.257800,1


Index(['Mean precision', 'Recall', 'F-Measure'], dtype='object')

In [4]:
# display top f_measure scores of each batch
index=2
with pd.option_context('display.max_rows', None):  # more options can be specified also
    display(results_df.groupby("Batch").apply(lambda x: x.nlargest(25, metrics[index])).drop(columns=["Batch"]))

System  Mean precision    Recall  F-Measure
Batch                                                               
1     1        bio-answerfinder        0.390800  0.417000   0.355300
      0                Proposed        0.305556  0.540372   0.312666
      2                  RYGH-1        0.288900  0.612200   0.299900
      3                  RYGH-4        0.277400  0.617700   0.294300
      4                  RYGH-3        0.276500  0.616200   0.293700
      5                    RYGH        0.273000  0.598000   0.291100
      6    The basic end-to-end        0.249600  0.513500   0.278700
      7     Basic e2e mid speed        0.239600  0.496000   0.266800
      8             gsl_zs_rrf1        0.231100  0.565800   0.258400
      9      bio-answerfinder-2        0.261300  0.471500   0.257800
      10            gsl_zs_rrf2        0.228900  0.552900   0.255000
      11              bioinfo-1        0.231100  0.556900   0.253900
      12              bioinfo-0        0.228900  0.557400   0.253200
      13              bioinfo-3        0.228900  0.552900   0.250800
      14              bioinfo-2        0.225600  0.555000   0.250400
      15          gsl_zs_hybrid        0.222200  0.551400   0.248200
      16            gsl_zs_rrf3        0.220000  0.516700   0.244900
      17              gsl_zs_nn        0.206700  0.526900   0.231400
      18           AUEB-System2        0.210000  0.463400   0.228000
      19           AUEB-System1        0.167800  0.409200   0.189900
      20         ELECTROLBERT-0        0.087800  0.159000   0.086600
      21         ELECTROLBERT-1        0.087800  0.159600   0.086200
      22         ELECTROLBERT-2        0.087800  0.159600   0.086200
      23   simple baseline solr        0.002200  0.001500   0.001800
2     25       bio-answerfinder        0.345800  0.406100   0.309400
      24               Proposed        0.292222  0.523762   0.282079
      26                 RYGH-3        0.285900  0.572200   0.280500
      27                 RYGH-4        0.286800  0.568600   0.279200
      28                   RYGH        0.282500  0.563200   0.274300
      29                 RYGH-1        0.275800  0.580900   0.273600
      30     bio-answerfinder-2        0.281100  0.411200   0.257300
      31     bio-answerfinder-4        0.275300  0.396800   0.249800
      32     bio-answerfinder-3        0.292400  0.309000   0.245000
      33            gsl_zs_rrf1        0.224400  0.511600   0.231000
      34          gsl_zs_hybrid        0.223300  0.517600   0.229900
      35                 RYGH-5        0.230000  0.482400   0.227400
      36            gsl_zs_rrf2        0.214400  0.509200   0.222800
      37            gsl_zs_rrf3        0.216700  0.486600   0.219700
      38              gsl_zs_nn        0.205600  0.503200   0.216200
      39              bioinfo-3        0.206700  0.476700   0.213500
      40              bioinfo-0        0.206700  0.465800   0.212800
      41              bioinfo-2        0.206700  0.466700   0.212800
      42              bioinfo-1        0.205600  0.465000   0.212000
      43           AUEB-System2        0.206700  0.471500   0.206000
      44    Basic e2e mid speed        0.185900  0.443500   0.204600
      45   The basic end-to-end        0.184800  0.440800   0.203200
      46              bioinfo-4        0.178900  0.408600   0.184900
      47    Deep ML methods for        0.172200  0.434300   0.180100
      48      MindLab QA System        0.172200  0.434300   0.180100
3     57       bio-answerfinder        0.376700  0.482600   0.346300
      58   The basic end-to-end        0.309300  0.514800   0.310200
      59      BioNIR Prepro-mid        0.307000  0.507000   0.308100
      60    Basic e2e mid speed        0.304700  0.513100   0.307500
      61     bio-answerfinder-3        0.337800  0.431700   0.307300
      56               Proposed        0.294382  0.576637   0.303103
      62                 RYGH-5        0.256700  0.657400   0.286100
      63               

In [ ]:
# comparison of each batch
for batch in results_df['batch'].unique():
    # Choose the batch number for which you want to visualize the data
    selected_batch = batch

    # Filter the data for the selected batch
    selected_batch_data = results_df[results_df['batch'] == selected_batch]

    # Melt the dataframe to combine all metrics into a single column
    melted_data = pd.melt(selected_batch_data, id_vars=['system'], value_vars=metrics)

    # Set the bright color palette
    sns.set_palette("bright")

    # Create a grouped bar plot to visualize the comparison of all metrics for each system
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='system', y='value', hue='variable', data=melted_data, errorbar=None)

    # Display numeric values on top of each bar
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    # Set the title, x-axis label, and y-axis label
    plt.xlabel('System')
    plt.ylabel('Score')
    title = f"[{challenge}] Comparison of Systems by Each Team (Batch {selected_batch})"
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # save the plot as png
    plt.savefig(f"{RESULTS_DIR}/{title}.png")
    # Show the plot
    plt.show()


In [ ]:
# Set the bright color palette
sns.set_palette("bright")

# Get unique batch numbers
unique_batches = results_df['batch'].unique()

# Create a grid of plots
num_cols = 3  # Number of columns in the grid
num_rows = -(-len(unique_batches) // num_cols)  # Calculate the number of rows needed

fig, axes = plt.subplots(num_rows, num_cols, figsize=(3*10, 7 * num_rows))

for i, batch in enumerate(unique_batches):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axes[row_idx, col_idx]

    # Filter the data for the selected batch
    selected_batch_data = results_df[results_df['batch'] == batch]

    # Melt the dataframe to combine all metrics into a single column
    melted_data = pd.melt(selected_batch_data, id_vars=['system'], value_vars=metrics)

    # Create a grouped bar plot to visualize the comparison of all metrics for each system
    sns.barplot(x='system', y='value', hue='variable', data=melted_data, ax=ax, errorbar=None)

    # Display numeric values on top of each bar
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    # Set the title, x-axis label, and y-axis label
    ax.set_xlabel('System')
    ax.set_ylabel('Score')
    ax.set_title(f'[{challenge}] Batch {batch}')

    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.legend(title='Metrics')

# Adjust layout
plt.tight_layout()

# save the plot as png
title = f'[{challenge}] Comparison of Systems by Team (all in one)'
plt.savefig(f"{RESULTS_DIR}/{title}.png")

# Show the plot
plt.show()


In [ ]:
# groupby each system and calculate the mean of each metric
data_grouped = results_df.groupby(['system'])[metrics].mean()
# sort by 'f_measure' column
data_grouped.sort_values(by=metrics[2], ascending=False)

In [ ]:
# Melt the dataframe to combine all metrics into a single column
melted_data = pd.melt(results_df, id_vars=['system', 'batch'], value_vars=metrics)

# Set the bright color palette
sns.set_palette("bright")

# Create a grouped bar plot to visualize the comparison of all metrics for each system
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='system', y='value', hue='variable', data=melted_data, errorbar=None)

# Display numeric values on top of each bar
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.xlabel('System')
plt.ylabel('Score')
title = f"[{challenge}] Comparison of Systems by Each Team (Average of All Batches)"
plt.title(title)
plt.xticks(rotation=45)
plt.tight_layout()

# save the plot
plt.savefig(f"{RESULTS_DIR}/{title}.png")

# Show the plot
plt.show()

In [ ]:
# Melt the dataframe to combine all metrics into a single column
melted_data = pd.melt(results_df, id_vars=['system', 'batch'], value_vars=metrics)

# Set the bright color palette
sns.set_palette("bright")

# Create a line plot to visualize the trends over batches for each system and metric
plt.figure(figsize=(12, 6))
sns.lineplot(x='batch', y='value', hue='variable', style='system', data=melted_data, markers=True)
plt.xlabel('Batch')
plt.ylabel('Score')
title = f"[{challenge}] Trends over Batches"
plt.title(title)
plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0.)
plt.tight_layout()

# save the plot
plt.savefig(f"{RESULTS_DIR}/{title}.png")

# Show the plot
plt.show()

In [ ]:
# Find the top 3 systems based on metric for the entire dataset
metric = 'f_measure'
top_3_systems = results_df.groupby('system')[metric].mean().nlargest(3).index

# Filter the data to include only the top 3 systems
top_3_data = results_df[results_df['system'].isin(top_3_systems)]

# Melt the dataframe to combine all metrics into a single column
melted_data = pd.melt(top_3_data, id_vars=['system', 'batch'], value_vars=['mean_precision', 'recall', 'f_measure'])

# Set the bright color palette
sns.set_palette("bright")

# Create a line plot to visualize the trends over batches for the top 3 systems and metrics
plt.figure(figsize=(12, 6))
sns.lineplot(x='batch', y='value', hue='variable', style='system', data=melted_data, markers=True)
plt.xlabel('Batch')
plt.ylabel('Score')
title = f"[{challenge}]Trends over Batches for Top 3 Systems (Based on {metric})"
plt.title(title)
plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0.)
plt.tight_layout()

#save the plot
plt.savefig(f"{RESULTS_DIR}/{title}.png")

# Show the plot
plt.show()

# Answer Extraction

In [ ]:
try:
    # load dir_dict from json file in home directory
    home_dir = os.path.expanduser("~")
    with open(f"{home_dir}/.biomedqa_dir.json", encoding="utf-8") as fp:
        dir_dict = json.load(fp)
except Exception as exc:
    print("Error: unable to load directory dictionary. Please run setup.py")
    raise exc

# set directories
BASE_DIR = dir_dict["base_dir"]
DATA_DIR = dir_dict["data_dir"]
MODEL_DIR = dir_dict["model_dir"]
LOG_DIR = dir_dict["log_dir"]
RESULTS_DIR = dir_dict["results_dir"]

# set parameters
challenge_no = 10
DATASET = 'bioasq'
YEAR = '2022'
DATASET_NAME = 'Task10BGoldenEnriched'
q_type = 'list'
RESULTS_DIR = f"{RESULTS_DIR}/biomed_qa/answer_extraction/transformer/minilm_ft/{DATASET}/{YEAR}/{DATASET_NAME}"
TEAM_SCORE_FILE = f"{RESULTS_DIR}/{DATASET_NAME}_{q_type}_results_by_team.csv"

# Load team scores into a DataFrame
team_scores_df = pd.read_csv(TEAM_SCORE_FILE)

# remove rows where system == proposed
team_scores_df = team_scores_df[team_scores_df['system'] != 'proposed']

# load proposed scores in to a DataFrame
model = 'list_model_15'
context_scr = 'by_snippets'
PROPOSED_SCORE_DIR = f"{RESULTS_DIR}/{model}/{context_scr}"
PROPOSED_SCORE_FILE = f"{PROPOSED_SCORE_DIR}/{DATASET_NAME}_{q_type}_results.csv"
PROPOSED_SCORE_FILE = f"{PROPOSED_SCORE_DIR}/{DATASET_NAME}_{q_type}_results.csv"
proposed_scores_df = pd.read_csv(PROPOSED_SCORE_FILE)

# add system and phase columns
proposed_scores_df['system'] = 'Proposed'
proposed_scores_df['phase'] = 'A & B'
score_df = pd.concat([team_scores_df, proposed_scores_df], ignore_index=True, sort=False)
score_df = proposed_scores_df
display(score_df)

# set challenge & metrics
challenge = f"BioASQ Task {challenge_no}b - Phase B"
metrics = score_df.columns[2:]
display(metrics)

In [16]:
try:
    # load dir_dict from json file in home directory
    home_dir = os.path.expanduser("~")
    with open(f"{home_dir}/.biomedqa_dir.json", encoding="utf-8") as fp:
        dir_dict = json.load(fp)
except Exception as exc:
    print("Error: unable to load directory dictionary. Please run setup.py")
    raise exc

# set directories
BASE_DIR = dir_dict["base_dir"]
DATA_DIR = dir_dict["data_dir"]
MODEL_DIR = dir_dict["model_dir"]
LOG_DIR = dir_dict["log_dir"]
RESULTS_DIR = dir_dict["results_dir"]

# set parameters
challenge_no = 10
DATASET = 'bioasq'
YEAR = '2022'
DATASET_NAME = 'Task10BGoldenEnriched'
q_type = 'list'
RESULTS_DIR = f"{RESULTS_DIR}/biomed_qa/answer_extraction/transformer/minilm_ft/{DATASET}/{YEAR}/{DATASET_NAME}"
scores_df = pd.read_csv(f"{RESULTS_DIR}/{DATASET_NAME}_factoid_and_list.csv")
display(score_df)

# set challenge & metrics
challenge = f"BioASQ Task {challenge_no}b - Phase B"
metrics = score_df.columns[2:]
display(metrics)

,System,Batch,factoid_strict,factoid_lenient,factoid_mrr,list_mean_prec,list_recall,list_F-measure
0,Proposed(golden_snippets),1,0.6176,0.8823,0.7132,0.79676,0.6511,0.66143
1,Proposed(golden_documents),1,0.4411,0.7941,0.5696,0.52710,0.7134,0.55940
2,Proposed(document_retrieval),1,0.3823,0.6764,0.4828,0.23860,0.6380,0.37920
3,UDEL-LAB5,1,0.3824,0.5882,0.4706,0.47140,0.8310,0.58100
4,Ir_sys1,1,0.4118,0.5000,0.4559,0.57140,0.4702,0.48870
...,...,...,...,...,...,...,...,...
170,AUEB-System4,6,0.1667,0.5000,0.3056,0.34670,0.1491,0.19120
171,Ir_sys3,6,0.1667,0.5000,0.2917,0.58520,0.3380,0.37400
172,NCU-IISR-AS-GIS-5,6,0.1667,0.3333,0.2222,0.69330,0.3108,0.34580
173,NCU-IISR/AS-GIS-3,6,0.1667,0.3333,0.2222,0.69330,0.3108,0.34580


Index(['factoid_strict', 'factoid_lenient', 'factoid_mrr', 'list_mean_prec',
       'list_recall', 'list_F-measure'],
      dtype='object')

In [18]:
# display top x metric scores of each batch
index = 2
print(f"Top {metrics[index]} scores of each batch:")
with pd.option_context('display.max_rows', None):  # more options can be specified also
    display(score_df.groupby("Batch").apply(lambda x: x.nlargest(35, metrics[index])).drop(columns=["Batch"]))

Top factoid_mrr scores of each batch:


System  factoid_strict  factoid_lenient  \
Batch                                                                      
1     0       Proposed(golden_snippets)          0.6176           0.8823   
      1      Proposed(golden_documents)          0.4411           0.7941   
      2    Proposed(document_retrieval)          0.3823           0.6764   
      3                       UDEL-LAB5          0.3824           0.5882   
      4                         Ir_sys1          0.4118           0.5000   
      5                         Ir_sys3          0.4118           0.5294   
      6                       UDEL-LAB2          0.3824           0.5588   
      7                       UDEL-LAB1          0.3529           0.5882   
      8                          lalala          0.3824           0.5000   
      9                         Ir_sys2          0.3529           0.5000   
      10              KU-AAA637-system1          0.3529           0.5294   
      11                          LaRSA          0.2941           0.5588   
      13             bio-answerfinder-2          0.3824           0.4118   
      12               bio-answerfinder          0.3824           0.4118   
      14                      UDEL-LAB4          0.3235           0.4412   
      15                      UDEL-LAB3          0.2941           0.4412   
      16             Knowledge graph QA          0.2647           0.4412   
      17                   AUEB-System4          0.2941           0.4118   
      18                   AUEB-System2          0.2941           0.3824   
      19                   AUEB-System1          0.2941           0.3529   
      20                   AUEB-System3          0.2941           0.3824   
      21                        Ir_sys4          0.2353           0.4118   
      22           simple baseline solr          0.2941           0.2941   
      23                     orpheus_kg          0.1471           0.1471   
      24                BioASQ_Baseline          0.0588           0.2647   
2     26     Proposed(golden_documents)          0.7352           0.8529   
      25      Proposed(golden_snippets)          0.6764           0.8235   
      28                      UDEL-LAB3          0.5882           0.6765   
      29                      UDEL-LAB4          0.5882           0.7059   
      30                      UDEL-LAB2          0.5882           0.6471   
      31                      UDEL-LAB1          0.5882           0.6471   
      32                        Ir_sys3          0.5882           0.6471   
      33                         lalala          0.5882           0.6471   
      34                        Ir_sys4          0.5882           0.6176   
      35              KU-AAA637-system4          0.5588           0.6471   
      36                        Ir_sys2          0.5588           0.6176   
      37              KU-AAA637-system5          0.5588           0.6176   
      38                        Ir_sys1          0.5588           0.6176   
      39              KU-AAA637-system1          0.5294           0.6471   
      40              KU-AAA637-system3          0.5294           0.6471   
      41              KU-AAA637-system2          0.5294           0.6176   
      42                          LaRSA          0.4412           0.5882   
      43               bio-answerfinder          0.4118           0.4706   
      44             bio-answerfinder-2          0.4118           0.4706   
      45                   AUEB-System1          0.3824           0.5000   
      46                   AUEB-System3          0.4118           0.4706   
      47           simple baseline solr          0.4118           0.4118   
      48                   AUEB-System2          0.3235           0.5000   
      27   Proposed(document_retrieval)          0.2549           0.6078   
      49                   AUEB-System4          0.2647           0.4706   
      50                     orpheus_kg          0.3529           0.3529   
     

In [ ]:
# comparison of each batch
for batch in score_df['batch'].unique():
    # Choose the batch number for which you want to visualize the data
    selected_batch = batch

    # Filter the data for the selected batch
    selected_batch_data = score_df[score_df['batch'] == selected_batch]

    # Melt the dataframe to combine all metrics into a single column
    melted_data = pd.melt(selected_batch_data, id_vars=['system'], value_vars=metrics)

    # Set the bright color palette
    sns.set_palette("bright")

    # Create a grouped bar plot to visualize the comparison of all metrics for each system
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='system', y='value', hue='variable', data=melted_data, errorbar=None)

    # Display numeric values on top of each bar
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    # Set the title, x-axis label, and y-axis label
    plt.xlabel('System')
    plt.ylabel('Score')
    title = f'[{challenge}] Comparison of Systems by Team for {q_type} questions Batch {selected_batch} (context={context_scr})'
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # save the plot as png
    plt.savefig(f"{PROPOSED_SCORE_DIR}/{title}.png")
    # Show the plot
    plt.show()

In [ ]:
# Set the bright color palette
sns.set_palette("bright")

# Get unique batch numbers
unique_batches = score_df['batch'].unique()

# Create a grid of plots
num_cols = 3  # Number of columns in the grid
num_rows = -(-len(unique_batches) // num_cols)  # Calculate the number of rows needed

fig, axes = plt.subplots(num_rows, num_cols, figsize=(3*10, 7 * num_rows))

for i, batch in enumerate(unique_batches):
    row_idx = i // num_cols
    col_idx = i % num_cols
    ax = axes[row_idx, col_idx]

    # Filter the data for the selected batch
    selected_batch_data = score_df[score_df['batch'] == batch]

    # Melt the dataframe to combine all metrics into a single column
    melted_data = pd.melt(selected_batch_data, id_vars=['system'], value_vars=metrics)

    # Create a grouped bar plot to visualize the comparison of all metrics for each system
    sns.barplot(x='system', y='value', hue='variable', data=melted_data, ax=ax, errorbar=None)

    # Display numeric values on top of each bar
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    # Set the title, x-axis label, and y-axis label
    ax.set_xlabel('System')
    ax.set_ylabel('Score')
    ax.set_title(f'[{challenge}] Batch {batch}')

    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    ax.legend(title='Metrics')

# Adjust layout
plt.tight_layout()

# save the plot as png
title = f'[{challenge}] Comparison of Systems by Team for {q_type} questions all in one (context={context_scr})'
plt.savefig(f"{PROPOSED_SCORE_DIR}/{title}.png")
# Show the plot
plt.show()


In [ ]:
# groupby each system and calculate the mean of each metric
# data_grouped = score_df.groupby(['system']).mean()
data_grouped = score_df.groupby(['system'])[metrics].mean()
# sort by x metric column
index = 2
print(f"sorted by {metrics[index]} scores:")
data_grouped.sort_values(by=metrics[index], ascending=False)

In [ ]:
# Melt the dataframe to combine all metrics into a single column
melted_data = pd.melt(score_df, id_vars=['system', 'batch'], value_vars=metrics)

# Set the bright color palette
sns.set_palette("bright")

# Create a grouped bar plot to visualize the comparison of all metrics for each system
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='system', y='value', hue='variable', data=melted_data, errorbar=None)

# Display numeric values on top of each bar
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.xlabel('System')
plt.ylabel('Score')
# title = f'[{challenge}] Comparison of Systems by Team for {q_type} questions Average of All Batches (context={context_scr})'
title = f'[{challenge}] Comparison of Systems by Team for {q_type} questions Average of All Batches (Golden snippets as input)'
plt.title(title)
plt.xticks(rotation=45)
plt.tight_layout()

# save the plot
plt.savefig(f"{PROPOSED_SCORE_DIR}/{title}.png")
# Show the plot
plt.show()

In [ ]:
both_phase_systems = score_df[score_df['phase'] == 'A & B']
# display top x metric scores of each batch
index = 2
print(f"Top {metrics[index]} scores of each batch:")
display(both_phase_systems.groupby("batch").apply(lambda x: x.nlargest(3, metrics[2])))

In [ ]:
if context_scr == 'by_lda':
    both_phase_systems = score_df[score_df['phase'] == 'A & B']
    # Melt the dataframe to combine all metrics into a single column
    melted_data = pd.melt(both_phase_systems, id_vars=['system', 'batch'], value_vars=metrics)

    # Set the bright color palette
    sns.set_palette("bright")

    # Create a grouped bar plot to visualize the comparison of all metrics for each system
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x='system', y='value', hue='variable', data=melted_data, errorbar=None)

    # Display numeric values on top of each bar
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    plt.xlabel('System')
    plt.ylabel('Score')
    title = f'[{challenge}] Systems participated in A & B for {q_type} questions Average of All Batches (context={context_scr})'
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # save the plot
    plt.savefig(f"{PROPOSED_SCORE_DIR}/{title}.png")
    # Show the plot
    plt.show()